In [1]:
!pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Import Libraries

In [2]:
import mysql.connector as mysql
import pandas as pd

## DB Credentials

In [3]:
db_host = "database-1.cxbcxwezo94b.us-east-1.rds.amazonaws.com"
db_name = 'ibm_database'
db_user = 'admin'
db_pass = 'Passw0rd123'

## Database Connection Function

In [18]:
# Conection
def connect_database(host, username, password, database):
    """
    Connect to database
    Params:
        - host: database host
        - username: database user
        - password: database password
        - database: database name
        
    Returns
        conn: connection with the db
    """
    try:
        # connection function
        conn = mysql.connect(host=host, user=username, password=password, db=database, auth_plugin='mysql_native_password')
        
        print('Connected to database')
        
        return conn
        
    # handles exceptions
    except Exception as e:
        print('Connection failed! Exception: ', e)


## Call Function Connection DB

In [19]:
# call function connect_database()
conn = connect_database(db_host, db_user, db_pass, db_name)

Connected to database


# Joins Exercises

### Question 1: Write and execute a SQL query to list the school names, community names and average attendance for communities with a hardship index of 98.

In [9]:
%%capture --no-display

pd.read_sql("""
    SELECT
        S.NAME_OF_SCHOOL,
        S.COMMUNITY_AREA_NAME,
        S.AVERAGE_STUDENT_ATTENDANCE
    FROM CHICAGO_PUBLIC_SCHOOLS S
    LEFT JOIN CENSUS_DATA C
    ON C.COMMUNITY_AREA_NUMBER = S.COMMUNITY_AREA_NUMBER
    WHERE C.HARDSHIP_INDEX = 98;
""", con=conn)

,NAME_OF_SCHOOL,COMMUNITY_AREA_NAME,AVERAGE_STUDENT_ATTENDANCE
0,George Washington Carver Military Academy High...,RIVERDALE,91.60%
1,George Washington Carver Primary School,RIVERDALE,90.90%
2,Ira F Aldridge Elementary School,RIVERDALE,92.90%
3,William E B Dubois Elementary School,RIVERDALE,93.30%


### Question 2: Write and execute a SQL query to list all crimes that took place at a school. Include case number, crime type and community name.

In [12]:
%%capture --no-display

pd.read_sql("""
    SELECT
        CR.CASE_NUMBER,
        CR.PRIMARY_TYPE,
        CE.COMMUNITY_AREA_NAME
    FROM CHICAGO_CRIME_DATA CR
    JOIN CENSUS_DATA CE
    ON CR.COMMUNITY_AREA_NUMBER = CE.COMMUNITY_AREA_NUMBER
    WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%'
    ORDER BY CR.PRIMARY_TYPE;
""", con=conn)

,CASE_NUMBER,PRIMARY_TYPE,COMMUNITY_AREA_NAME
0,HT315369,ASSAULT,East Garfield Park
1,HL725506,BATTERY,Lincoln Square
2,HH639427,BATTERY,Austin
3,HP716225,BATTERY,Douglas
4,HL353697,BATTERY,South Shore
5,JA460432,BATTERY,Ashburn
6,HS200939,CRIMINAL DAMAGE,Austin
7,HR585012,CRIMINAL TRESPASS,Ashburn
8,HK577020,NARCOTICS,Rogers Park
9,HS305355,NARCOTICS,Brighton Park


# Views Exercises

### Question 1: Write and execute a SQL statement to create a view showing the columns listed in the following table, with new column names as shown in the second column.

| Column name in CHICAGO_PUBLIC_SCHOOLS	| Column name in view |
| --- | --- |
| NAME_OF_SCHOOL	| School_Name |
| Safety_Icon	| Safety_Rating |
| Family_Involvement_Icon |	Family_Rating |
| Environment_Icon	| Environment_Rating |
| Instruction_Icon	| Instruction_Rating |
| Leaders_Icon	| Leaders_Rating |
| Teachers_Icon	| Teachers_Rating |

In [13]:
%%capture --no-display

pd.read_sql("""
    CREATE OR REPLACE VIEW v_public_schools 
(School_Name, Safety_Rating, Family_Rating, Environment_Rating, Instruction_Rating, Leaders_Rating, Teachers_Rating)
    AS SELECT
        NAME_OF_SCHOOL,
        Safety_Icon,
        Family_Involvement_Icon,
        Environment_Icon,
        Instruction_Icon,
        Leaders_Icon,
        Teachers_Icon
    FROM CHICAGO_PUBLIC_SCHOOLS;
""", con=conn)

TypeError: 'NoneType' object is not iterable

### Question 2: Write and execute a SQL statement that returns all of the columns from the view.

In [27]:
%%capture --no-display

pd.read_sql("""
    SELECT * FROM v_public_schools
""", con=conn)

,School_Name,Safety_Rating,Family_Rating,Environment_Rating,Instruction_Rating,Leaders_Rating,Teachers_Rating
0,Abraham Lincoln Elementary School,Very Strong,Very Strong,Strong,Strong,Average,Strong
1,Adam Clayton Powell Paideia Community Academy ...,Average,Strong,Strong,Very Strong,Weak,Strong
2,Adlai E Stevenson Elementary School,Strong,NDA,Average,Weak,Weak,NDA
3,Agustin Lara Elementary Academy,Average,Average,Average,Weak,Weak,Average
4,Air Force Academy High School,Average,Strong,Strong,Average,Weak,Average
...,...,...,...,...,...,...,...
561,William T Sherman Elementary School,Weak,NDA,Average,Average,Weak,NDA
562,William W Carter Elementary School,Very Weak,Average,Weak,Weak,Weak,Strong
563,Wolfgang A Mozart Elementary School,Average,NDA,Average,Weak,Weak,NDA
564,Woodlawn Community Elementary School,Strong,NDA,Very Strong,Strong,Weak,NDA


### Question 3: Write and execute a SQL statement that returns just the school name and leaders rating from the view.

In [20]:
%%capture --no-display

pd.read_sql("""
    SELECT
        School_Name,
        Leaders_Rating
    FROM v_public_schools;
""", con=conn)

,School_Name,Leaders_Rating
0,Abraham Lincoln Elementary School,Average
1,Adam Clayton Powell Paideia Community Academy ...,Weak
2,Adlai E Stevenson Elementary School,Weak
3,Agustin Lara Elementary Academy,Weak
4,Air Force Academy High School,Weak
...,...,...
561,William T Sherman Elementary School,Weak
562,William W Carter Elementary School,Weak
563,Wolfgang A Mozart Elementary School,Weak
564,Woodlawn Community Elementary School,Weak


### Question 1: Write the structure of a query to create or replace a stored procedure called UPDATE_LEADERS_SCORE that takes a in_School_ID parameter as an integer and a in_Leader_Score parameter as an integer. Don’t forget to use the #SET TERMINATOR statement to use the @ for the CREATE statement terminator.

In [ ]:
--#SET TERMINATOR @
CREATE PROCEDURE UPDATE_LEADERS_SCORE (
    IN in_School_ID INTEGER, in_Leader_Score INTEGER) 
LANGUAGE SQL 
MODIFIES SQL DATA
BEGIN
    
END 
  @

### Question 2: Inside your stored procedure, write a SQL statement to update the Leaders_Score field in the CHICAGO_PUBLIC_SCHOOLS table for the school identified by in_School_ID to the value in the in_Leader_Score parameter

In [ ]:
--#SET TERMINATOR @
CREATE PROCEDURE UPDATE_LEADERS_SCORE (
    IN p_school_id INTEGER, p_leader_score INTEGER) 
LANGUAGE SQL 
MODIFIES SQL DATA
BEGIN
    UPDATE "CHICAGO_PUBLIC_SCHOOLS"
    SET "Leaders_Score" = p_leader_score
    WHERE "School_ID" = p_school_id;
END 
@

### Question 3: Inside your stored procedure, write a SQL IF statement to update the Leaders_Icon field in the CHICAGO_PUBLIC_SCHOOLS table for the school identified by in_School_ID using the following information.

| Score lower limit | Score upper limit | Icon |
| --- | --- | --- |
| 80 | 99 | Very strong |
| 60 | 79 | Strong |
| 40 | 59 | Average |
| 20 | 39 | Weak |
| 0	| 19 | Very weak |

In [ ]:
DELIMITER $$

CREATE PROCEDURE `p_update_leaders_score`(IN p_school_id INTEGER, IN p_leader_score INTEGER)
BEGIN
	IF p_leader_score >= 80 THEN
		UPDATE CHICAGO_PUBLIC_SCHOOLS
        SET LEADERS_ICON = 'Very Strong', LEADERS_SCORE = p_leader_score
        WHERE SCHOOL_ID = p_school_id;
        
	ELSEIF p_leader_score >= 60 AND p_leader_score <=79 THEN
		UPDATE CHICAGO_PUBLIC_SCHOOLS
        SET LEADERS_ICON = 'Strong', LEADERS_SCORE = p_leader_score
        WHERE SCHOOL_ID = p_school_id;
        
	ELSEIF p_leader_score >=40 AND p_leader_score <= 59 THEN
		UPDATE CHICAGO_PUBLIC_SCHOOLS
        SET LEADERS_ICON = 'Average', LEADERS_SCORE = p_leader_score
        WHERE SCHOOL_ID = p_school_id;
        
	ELSEIF p_leader_score >= 20 AND p_leader_score <= 39 THEN
		UPDATE CHICAGO_PUBLIC_SCHOOLS
		SET LEADERS_ICON = 'Weak', LEADERS_SCORE = p_leader_score
        WHERE SCHOOL_ID = p_school_id;
        
	ELSE 
		UPDATE CHICAGO_PUBLIC_SCHOOLS
        SET LEADERS_ICON = 'Very Weak', LEADERS_SCORE = p_leader_score
        WHERE SCHOOL_ID = p_school_id;
		
	END IF;

END$$

DELIMITER ;

### Question 4: Write a query to call the stored procedure, passing a valid school ID and a leader score of 50, to check that the procedure works as expected.

In [ ]:
%%capture --no-display

pd.read_sql("""
    CALL p_update_leaders_score(610038, 50)
""", con=conn);

## Transactions

### Question 1: Update your stored procedure definition. Add a generic ELSE clause to the IF statement that rolls back the current work if the score did not fit any of the preceding categories.

## Close DB Connection

In [28]:
conn.close()
print('Connection Closed!')

Connection Closed!
